# MCP Solution: Building a Multi-Server Agent

This notebook contains the complete solution for building a LangGraph agent that integrates both filesystem and GitHub MCP servers.

In [ ]:
# Install mcp, langgraph, and langchain if needed (uncomment when running locally)
# !pip install langchain-mcp-adapters langgraph>=0.2.0 langchain-openai mcp

import asyncio
from langchain_openai import ChatOpenAI
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.graph import StateGraph, MessagesState, START, END
from langgraph.prebuilt import ToolNode

import os
## SET YOUR OPENAI_API_KEY HERE

## Complete Solution Implementation
/Users/gp/Udacity/udacity-cd14639-intro-langchain/lesson-1-ModelContextProtocol/exercises/solution/langgraph_github_fs_demo.py
This solution demonstrates how to:
1. Configure MCP server connections for filesystem and GitHub servers
2. Create a LangGraph agent that can use tools from both servers
3. Orchestrate fetching GitHub issues and saving summaries to files

In [ ]:
async def setup_graph() -> StateGraph:
    """
    Create and return a compiled LangGraph that integrates MCP tools.
    
    This is the complete solution showing proper MCP server configuration.
    """

    
    # SOLUTION: Define MCP server connections
    connections = {
        "filesystem": {
            "command": "python",
            "args": ["./filesystem_server.py"],
            "transport": "stdio",
        },
        "github": {
            "command": "python",
            "args": ["./github_server.py"],
            "transport": "stdio",
        },
    }
    
    # Create MCP client and get tools
    client = MultiServerMCPClient(connections)
    tools = await client.get_tools()
    
    # Initialize ChatOpenAI model and bind tools
    model = ChatOpenAI(model="gpt-4o", temperature=0)
    model_with_tools = model.bind_tools(tools)
    
    # Create ToolNode for executing tool calls
    tool_node = ToolNode(tools)
    
    # Define transition function for graph routing
    def should_continue(state: MessagesState):
        last = state["messages"][-1]
        if last.tool_calls:
            return "tools"
        return END
    
    # Define model invocation node
    async def call_model(state: MessagesState):
        messages = state["messages"]
        response = await model_with_tools.ainvoke(messages)
        return {"messages": [response]}
    
    # Build the state graph
    builder = StateGraph(MessagesState)
    builder.add_node("call_model", call_model)
    builder.add_node("tools", tool_node)
    builder.add_edge(START, "call_model")
    builder.add_conditional_edges("call_model", should_continue)
    builder.add_edge("tools", "call_model")
    
    return builder.compile()

## Solution Explanation

### MCP Server Configuration

The key part of this solution is the `connections` dictionary:

```python
connections = {
    "filesystem": {
        "command": "python",
        "args": ["./filesystem_server.py"],
        "transport": "stdio",
    },
    "github": {
        "command": "python",
        "args": ["./github_server.py"],
        "transport": "stdio",
    },
}
```

**Key Points:**
- Each server has a unique name ("filesystem", "github")
- `command`: The executable to run ("python" in both cases)
- `args`: List of arguments passed to the command (the server script paths)
- `transport`: Communication method ("stdio" for standard input/output)

### How it Works
1. `MultiServerMCPClient` uses these configurations to spawn the server processes
2. It communicates with each server via stdio transport
3. The client aggregates tools from both servers
4. LangGraph agent can then use any tool from either server seamlessly

In [19]:
async def run_example() -> None:
    """
    Run an example query against the graph.
    
    This demonstrates the complete working solution that fetches GitHub issues
    and saves summaries using the configured MCP servers.
    """
    # Set up the graph using the complete MCP configuration
    graph = await setup_graph()
    
    # Define the prompt for the task
    prompt = (
        "In the GitHub repository langchain-ai/langgraph (https://github.com/langchain-ai/langgraph), retrieve the most "
        "recently created issue. Provide a concise summary of the issue and "
        "append the summary to a file named 'issue_summary.txt' in the current "
        "working directory. Confirm when the summary has been saved."
    )
    
    # Invoke the graph with the prompt
    result = await graph.ainvoke({"messages": [{"role": "user", "content": prompt}]})
    
    # Print the final assistant message
    print(result["messages"][-1].content)

## Test the Complete Solution

Run the cell below to test the complete implementation:

In [20]:
# Test the complete solution
await run_example()

The summary of the most recent issue has been successfully appended to 'issue_summary.txt'.


## Expected Behavior

When running correctly, the solution will:

1. **Connect to MCP Servers**: Spawn both filesystem and GitHub server processes
2. **Fetch GitHub Issue**: Use GitHub server tools to retrieve the latest issue from langchain-ai/langgraph
3. **Process Content**: Analyze and summarize the issue content
4. **Save to File**: Use filesystem server tools to save the summary to `issue_summary.txt`
5. **Confirm Success**: Report that the task has been completed

The agent seamlessly orchestrates between the two MCP servers to complete the multi-step task.

## Key Learning Points

This solution demonstrates several important MCP concepts:

1. **Multi-Server Configuration**: How to connect to multiple MCP servers simultaneously
2. **Tool Aggregation**: Tools from different servers are combined into a single tool set
3. **Seamless Integration**: The agent doesn't need to know which server provides which tool
4. **Flexible Architecture**: Easy to add more servers by extending the connections dictionary
5. **Server Isolation**: Each server runs in its own process with defined boundaries

The MCP architecture allows for modular, scalable AI applications that can leverage specialized capabilities from different sources.